# Importando bibliotecas e data set

Baixando bibliotecas

## Importando bibliotecas

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from datetime import datetime

## Importando data sets

In [2]:
data_2020_2021_v2 = pd.read_csv("../dados/base_inteli_2020_2021_v2.csv", sep="," , encoding='utf8')
df_11 = pd.DataFrame(data_2020_2021_v2)

In [3]:
data_2022_2023_v2 = pd.read_csv("../dados/base_inteli_2022_2023_v2.csv", sep =",", encoding='utf8')
df_21 = pd.DataFrame(data_2022_2023_v2)

In [ ]:
df = pd.concat([df_11,df_21])

In [ ]:
data_2020_2021 = pd.read_csv("../dados/base_inteli 2020_2021.csv", sep=";" , encoding='utf8')
df_12 = pd.DataFrame(data_2020_2021)

data_2022_2023 = pd.read_csv("../dados/base_inteli_2022_2023.csv", sep =";", encoding='utf8')
df_22 = pd.DataFrame(data_2022_2023)

df_antiga = pd.concat([df_12,df_22])

df = df.merge(df_antiga[['sku', 'date', 'avg_website_visits_last_week','winning_price', 'price_status']], on=['sku', 'date'], how='left')

## Aplicação do dolar e da taxa SELIC no dataset

In [ ]:
dolar = pd.read_csv("../dados/dolar.csv", sep =";", encoding='utf8')
df_dolar = pd.DataFrame(dolar)

In [ ]:
df_dolar['data'] = pd.to_datetime(df_dolar['Data e hora da Cotacao']).dt.date


In [ ]:
df_dolar = df_dolar.drop(columns=['Data e hora da Cotacao'], axis=1)

In [ ]:
# Converte a string para um objeto datetime.date
date_threshold = datetime.strptime('2019-12-20', '%Y-%m-%d').date()
df_dolar = df_dolar[df_dolar['data'] > date_threshold]

In [ ]:
data_selic = pd.read_csv('../dados/selic.csv', sep = ";", encoding='utf8')
df_selic = pd.DataFrame(data_selic)

In [ ]:
df_selic['data'] = df_selic['data'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y').date())

In [ ]:
df_selic.rename(columns= {'valor': 'selic'}, inplace = True)

In [ ]:
date_threshold = datetime.strptime('2019-12-30', '%Y-%m-%d').date()
df_selic = df_selic[df_selic['data'] > date_threshold]

## Concatenando o data set em um dataframe

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df_dolar['data'] = pd.to_datetime(df_dolar['data'])

In [ ]:
df = pd.merge(df, df_dolar, how='left', left_on='date', right_on='data')

In [ ]:
df = df.drop(columns=['data'], axis=1)

In [ ]:
df_selic['data'] = pd.to_datetime(df_selic['data'])


In [ ]:
df = pd.merge(df, df_selic, how='left', left_on='date', right_on='data')

In [ ]:
num_nan = df['Cotacao Compra'].isnull().sum()

In [ ]:
df['Cotacao Compra'].fillna(method='bfill', inplace=True)

In [ ]:
df = df.drop(columns=['Cotacao Venda'], axis=1)
df = df.drop(columns=['data'], axis=1)

In [ ]:
df.rename(columns={'Cotacao Compra': 'dolar'}, inplace=True)

In [ ]:
df['dolar'] = df['dolar'].str.replace(',', '.').astype(float)

In [ ]:
num_nan = df['selic'].isnull().sum()

In [ ]:
df['selic'].fillna(method='ffill', inplace=True)
df['selic'].fillna(method='bfill', inplace=True)

In [ ]:
df['selic'] = df['selic'].str.replace(',', '.').astype(float)

Tratando a diferença do tamnaho do dataset df_comp e df

# Pré-Processamento

## Detecção de dados duplicados

In [ ]:
duplicados = df[df.duplicated(keep='first')]

## Eliminação de colunas desnecessárias





> Eliminamos a coluna mobly_item pois não interfere no nosso modelo.



In [ ]:
df = df.drop(columns=['mobly_item'])

Eliminamos a coluna weekday_name pois tratamos a data popsteriormente.

In [ ]:
df = df.drop(columns=['weekday_name'])

Eliminando a coluna de sales_type pois trata de dados redundantes.

In [ ]:
df = df.drop(columns=['sales_type'])

In [ ]:
df.region.isnull().sum()

In [ ]:
df.avg_customer_planned_delivery_workdays.isnull().sum()

In [ ]:
df.avg_customer_delivery_workdays.isnull().sum()

> Eliminamos as colunas relacionadas ao "bundle" devido a colinearidade com a coluna "items_sold".

In [ ]:
df = df.drop(columns=['flag_bundle'])

In [ ]:
df = df.drop(columns=['revenue_bundle'])

In [ ]:
df = df.drop(columns=['items_sold_bundle'])

> Eliminamos a coluna "supplier_delivery_time" pois não interfere no nosso modelo

In [ ]:
df = df.drop(columns=["supplier_delivery_time"])

> Eliminamos as colunas relacionadas às medidas do SKU pois não interfere no nosso modelo

In [ ]:
df = df.drop(columns=["sku_height"])

In [ ]:
df = df.drop(columns=["sku_width"])

In [ ]:
df = df.drop(columns=["sku_weight"])

In [ ]:
df = df.drop(columns=["sku_length"])

> Código para verificar o número de células vazias

In [ ]:
df.isnull().sum()

## Alterando colunas

## Preenchimento das colunas com lagging

In [ ]:
# inserir aqui (winning price e price_status)

## Codificação das variáveis categóricas

Verificação dos tipos das variáveis para identificar as categóricas que restaram após os tratamentos de colunas desnecessárias, dados faltantes e inconsistentes para serem codificadas.

In [ ]:
df.dtypes

Ferramenta para transformar variáveis categóricas não numéricas em uma representação numérica.

In [ ]:
le = LabelEncoder()

Codificação de variáveis categóricas com o LabelEncoder para variáveis que possuem mais de 6 valores únicos

In [ ]:
df['sku'] = le.fit_transform(df['sku'])

In [ ]:
df['anchor_category'] = le.fit_transform(df['anchor_category'])

In [ ]:
df['product_department'] = le.fit_transform(df['product_department'])

In [ ]:
df['product_category'] = le.fit_transform(df['product_category'])

In [ ]:
df['sku_color'] = le.fit_transform(df['sku_color'])

In [ ]:
df.region = le.fit_transform(df.region)

Codificação de variáveis categóricas com o get dummies do pandas (técnica do OneHotEncoder) para variáveis que possuem até 5 valores únicos

In [ ]:
df = pd.get_dummies(df, columns=['shipment_type', 'origin_country', 'process_costing'], drop_first='True')

## Tratamento data

Tratamento da váriavel date: Separação em outras colunas

In [ ]:
# Extraindo os componentes da data
df['ano'] = df['date'].dt.year
df['mes'] = df['date'].dt.month
df['dia'] = df['date'].dt.day

# 0: Segunda-feira, 1: Terça-feira, etc.
df['semana_do_ano'] = df['date'].dt.isocalendar().week
df['trimestre'] = df['date'].dt.quarter
df['dia_do_ano'] = df['date'].dt.dayofyear
df['dia_da_semana'] = df['date'].dt.weekday

# Retorna True para sábado e domingo
df['fim_de_semana'] = df['date'].dt.weekday >= 5  

# Transformação cíclica para o dia do ano
df['dia_do_ano_sin'] = np.sin(2 * np.pi * df['dia_do_ano']/365)
df['dia_do_ano_cos'] = np.cos(2 * np.pi * df['dia_do_ano']/365)

# Transformação cíclica para o dia da semana
df['dia_da_semana_sin'] = np.sin(2 * np.pi * df['dia_da_semana']/7)
df['dia_da_semana_cos'] = np.cos(2 * np.pi * df['dia_da_semana']/7)

In [ ]:
df.date = pd.to_datetime(df.date).dt.date

Codificação do atributo "year" com o get dummies do pandas (técnica do OneHotEncoder) para variáveis que possuem até 5 valores únicos

In [ ]:
df = pd.get_dummies(df, columns=['ano'], drop_first='True')

Agora a visualização das últimas 10 linhas do dataframe

In [ ]:
df.tail(10)

## Correção dummies

Tranformação das variáveis codificadas com dummies "True" e "False" em 1 e 0

In [ ]:
dummies_dict = {
    True: 1,
    False: 0
}

df = df.replace(dummies_dict)

# Armazenamento df

In [ ]:
df_2 = df

In [ ]:
%store df_2